# Pyds Module

In [1]:
"""
A framework for performing computations in the Dempster-Shafer theory.
"""

from __future__ import print_function
from itertools import chain, combinations
from functools import partial, reduce
from operator import mul
from math import log, fsum, sqrt
from random import random, shuffle, uniform
import sys

try:
    import numpy
    try:
        from scipy.stats import chi2
        from scipy.optimize import fmin_cobyla
    except:
        print('SciPy not found: some features will not work.', file=sys.stderr)
except:
    print('NumPy not found: some features will not work.', file=sys.stderr)


class MassFunction(dict):
    """
    A Dempster-Shafer mass function (basic probability assignment) based on a dictionary.
    
    Both normalized and unnormalized mass functions are supported.
    The underlying frame of discernment is assumed to be discrete.
    
    Hypotheses and their associated mass values can be added/changed/removed using the standard dictionary methods.
    Each hypothesis can be an arbitrary sequence which is automatically converted to a 'frozenset', meaning its elements must be hashable.
    """
    
    def __init__(self, source=None):
        """
        Creates a new mass function.
        
        If 'source' is not None, it is used to initialize the mass function.
        It can either be a dictionary mapping hypotheses to non-negative mass values
        or an iterable containing tuples consisting of a hypothesis and a corresponding mass value.
        """
        if source != None:
            if isinstance(source, dict):
                source = source.items()
            for (h, v) in source:
                self[h] += v
    
    @staticmethod
    def _convert(hypothesis):
        """Convert hypothesis to a 'frozenset' in order to make it hashable."""
        if isinstance(hypothesis, frozenset):
            return hypothesis
        else:
            return frozenset(hypothesis)
    
    @staticmethod
    def gbt(likelihoods, normalization=True, sample_count=None):
        """
        Constructs a mass function using the generalized Bayesian theorem.
        For more information, see Smets. 1993. Belief functions: 
        The disjunctive rule of combination and the generalized Bayesian theorem. International Journal of Approximate Reasoning. 
        
        'likelihoods' specifies the conditional plausibilities for a set of singleton hypotheses.
        It can either be a dictionary mapping singleton hypotheses to plausibilities or an iterable
        containing tuples consisting of a singleton hypothesis and a corresponding plausibility value.
        
        'normalization' determines whether the resulting mass function is normalized, i.e., whether m({}) == 0.
        
        If 'sample_count' is not None, the true mass function is approximated using the specified number of samples.
        """
        m = MassFunction()
        if isinstance(likelihoods, dict):
            likelihoods = list(likelihoods.items())
        # filter trivial likelihoods 0 and 1
        ones = [h for (h, l) in likelihoods if l >= 1.0]
        likelihoods = [(h, l) for (h, l) in likelihoods if 0.0 < l < 1.0]
        if sample_count == None:   # deterministic
            def traverse(m, likelihoods, ones, index, hyp, mass):
                if index == len(likelihoods):
                    m[hyp + ones] = mass
                else:
                    traverse(m, likelihoods, ones, index + 1, hyp + [likelihoods[index][0]], mass * likelihoods[index][1])
                    traverse(m, likelihoods, ones, index + 1, hyp, mass * (1.0 - likelihoods[index][1]))
            traverse(m, likelihoods, ones, 0, [], 1.0)
            if normalization:
                m.normalize()
        else:   # Monte-Carlo
            if normalization:
                empty_mass = reduce(mul, [1.0 - l[1] for l in likelihoods], 1.0)
            for _ in range(sample_count):
                rv = [random() for _ in range(len(likelihoods))]
                subtree_mass = 1.0
                hyp = set(ones)
                for k in range(len(likelihoods)):
                    l = likelihoods[k][1]
                    p_t = l * subtree_mass
                    p_f = (1.0 - l) * subtree_mass
                    if normalization and not hyp: # avoid empty hypotheses in the normalized case
                        p_f -= empty_mass
                    if p_t > rv[k] * (p_t + p_f):
                        hyp.add(likelihoods[k][0])
                    else:
                        subtree_mass *= 1 - l # only relevant for the normalized empty case
                m[hyp] += 1.0 / sample_count
        return m
    
    @staticmethod
    def from_bel(bel):
        """
        Creates a mass function from a corresponding belief function.
        
        'bel' is a dictionary mapping hypotheses to belief values (like the dictionary returned by 'bel(None)').
        """
        m = MassFunction()
        for h1 in bel.keys():
            v = fsum([bel[h2] * (-1)**(len(h1 - h2)) for h2 in powerset(h1)])
            if v > 0:
                m[h1] = v
        mass_sum = fsum(m.values())
        if mass_sum < 1.0:
            m[frozenset()] = 1.0 - mass_sum
        return m
    
    @staticmethod
    def from_pl(pl):
        """
        Creates a mass function from a corresponding plausibility function.
        
        'pl' is a dictionary mapping hypotheses to plausibility values (like the dictionary returned by 'pl(None)').
        """
        frame = max(pl.keys(), key=len)
        bel_theta = pl[frame]
        bel = {frozenset(frame - h):bel_theta - v for (h, v) in pl.items()} # follows from bel(-A) = bel(frame) - pl(A)
        return MassFunction.from_bel(bel)
    
    @staticmethod
    def from_q(q):
        """
        Creates a mass function from a corresponding commonality function.
        
        'q' is a dictionary mapping hypotheses to commonality values (like the dictionary returned by 'q(None)').
        """
        m = MassFunction()
        frame = max(q.keys(), key=len)
        for h1 in q.keys():
            v = fsum([q[h1 | h2] * (-1)**(len(h2 - h1)) for h2 in powerset(frame - h1)])
            if v > 0:
                m[h1] = v
        mass_sum = fsum(m.values())
        if mass_sum < 1.0:
            m[frozenset()] = 1.0 - mass_sum
        return m
    
    def __missing__(self, key):
        """Return 0 mass for hypotheses that are not contained."""
        return 0.0
    
    def __copy__(self):
        c = MassFunction()
        for k, v in self.items():
            c[k] = v
        return c
    
    def copy(self):
        """Creates a shallow copy of the mass function."""
        return self.__copy__()
    
    def __contains__(self, hypothesis):
        return dict.__contains__(self, MassFunction._convert(hypothesis))
    
    def __getitem__(self, hypothesis):
        return dict.__getitem__(self, MassFunction._convert(hypothesis))
    
    def __setitem__(self, hypothesis, value):
        """
        Adds or updates the mass value of a hypothesis.
        
        'hypothesis' is automatically converted to a 'frozenset' meaning its elements must be hashable.
        In case of a negative mass value, a ValueError is raised.
        """
        if value < 0.0:
            raise ValueError("mass value is negative: %f" % value)
        dict.__setitem__(self, MassFunction._convert(hypothesis), value)
    
    def __delitem__(self, hypothesis):
        return dict.__delitem__(self, MassFunction._convert(hypothesis))
    
    def frame(self):
        """
        Returns the frame of discernment of the mass function as a 'frozenset'.
        
        The frame of discernment is the union of all contained hypotheses.
        In case the mass function does not contain any hypotheses, an empty set is returned.
        """
        if not self:
            return frozenset()
        else:
            return frozenset.union(*self.keys())
    
    def singletons(self):
        """
        Returns the set of all singleton hypotheses.
        
        Like 'frame()', except that each singleton is wrapped in a frozenset
        and can thus be directly passed to methods like 'bel()'.
        """
        return {frozenset((s,)) for s in self.frame()}
    
    def focal(self):
        """
        Returns the set of all focal hypotheses.
        
        A focal hypothesis has a mass value greater than 0.
        """
        return {h for (h, v) in self.items() if v > 0}
    
    def core(self, *mass_functions):
        """
        Returns the core of one or more mass functions as a 'frozenset'.
        
        The core of a single mass function is the union of all its focal hypotheses.
        In case a mass function does not contain any focal hypotheses, its core is an empty set.
        If multiple mass functions are given, their combined core (intersection of all single cores) is returned.
        """
        if mass_functions:
            return frozenset.intersection(self.core(), *[m.core() for m in mass_functions])
        else:
            focal = self.focal()
            if not focal:
                return frozenset()
            else:
                return frozenset.union(*focal)
    
    def all(self):
        """Returns an iterator over all subsets of the frame of discernment, including the empty set."""
        return powerset(self.frame())
    
    def bel(self, hypothesis=None):
        """
        Computes either the belief of 'hypothesis' or the entire belief function (hypothesis=None).
        
        If 'hypothesis' is None (default), a dictionary mapping hypotheses to their respective belief values is returned.
        Otherwise, the belief of 'hypothesis' is returned.
        In this case, 'hypothesis' is automatically converted to a 'frozenset' meaning its elements must be hashable.
        """
        if hypothesis is None:
            return {h:self.bel(h) for h in powerset(self.core())}
        else:
            hypothesis = MassFunction._convert(hypothesis)
            if not hypothesis:
                return 0.0
            else:
                return fsum([v for (h, v) in self.items() if h and hypothesis.issuperset(h)])
    
    def pl(self, hypothesis=None):
        """
        Computes either the plausibility of 'hypothesis' or the entire plausibility function (hypothesis=None).
        
        If 'hypothesis' is None (default), a dictionary mapping hypotheses to their respective plausibility values is returned.
        Otherwise, the plausibility of 'hypothesis' is returned.
        In this case, 'hypothesis' is automatically converted to a 'frozenset' meaning its elements must be hashable.
        """
        if hypothesis is None:
            return {h:self.pl(h) for h in powerset(self.core())}
        else:
            hypothesis = MassFunction._convert(hypothesis)
            if not hypothesis:
                return 0.0
            else:
                return fsum([v for (h, v) in self.items() if hypothesis & h])
    
    def q(self, hypothesis=None):
        """
        Computes either the commonality of 'hypothesis' or the entire commonality function (hypothesis=None).
        
        If 'hypothesis' is None (default), a dictionary mapping hypotheses to their respective commonality values is returned.
        Otherwise, the commonality of 'hypothesis' is returned.
        In this case, 'hypothesis' is automatically converted to a 'frozenset' meaning its elements must be hashable.
        """
        if hypothesis is None:
            return {h:self.q(h) for h in powerset(self.core())}
        else:
            if not hypothesis:
                return 1.0
            else:
                return fsum([v for (h, v) in self.items() if h.issuperset(hypothesis)])
    
    def __and__(self, mass_function):
        """Shorthand for 'combine_conjunctive(mass_function)'."""
        return self.combine_conjunctive(mass_function)
    
    def __or__(self, mass_function):
        """Shorthand for 'combine_disjunctive(mass_function)'."""
        return self.combine_disjunctive(mass_function)
    
    def __str__(self):
        hyp = sorted([(v, h) for (h, v) in self.items()], reverse=True)
        return "{" + "; ".join([str(set(h)) + ":" + str(v) for (v, h) in hyp]) + "}"
    
    def __mul__(self, scalar):
        if not isinstance(scalar, float):
            raise TypeError('Can only multiply by a float value.')
        m = MassFunction()
        for (h, v) in self.items():
            m[h] = v * scalar
        return m
    
    def __rmul__(self, scalar):
        return self.__mul__(scalar)
    
    def __add__(self, m):
        if not isinstance(m, MassFunction):
            raise TypeError('Can only add two mass functions.')
        result = self.copy()
        for (h, v) in m.items():
            result[h] += v
        return result
    
    def weight_function(self):
        """
        Computes the weight function corresponding to this mass function.
        """
        weights = dict()
        q = self.q()
        theta = self.frame()
        for h in powerset(theta):
            if len(h) < len(theta): # weight is undefined for theta
                sets = [h | c for c in powerset(theta - h)]
                q_even = reduce(mul, [q[h2] for h2 in sets if len(h2) % 2 == 0], 1.0)
                q_odd = reduce(mul, [q[h2] for h2 in sets if len(h2) % 2 == 1], 1.0)
                if len(h) % 2 == 0:
                    weights[h] = q_odd / q_even
                else:
                    weights[h] = q_even / q_odd
        return weights
    
    def combine_conjunctive(self, mass_function, normalization=True, sample_count=None, importance_sampling=False):
        """
        Conjunctively combines the mass function with another mass function and returns the combination as a new mass function.
        
        The other mass function is assumed to be defined over the same frame of discernment.
        If 'mass_function' is not of type MassFunction, it is assumed to be an iterable containing multiple mass functions that are iteratively combined.
        
        If the mass functions are flatly contracting or if one of the mass functions is empty, an empty mass function is returned.
        
        'normalization' determines whether the resulting mass function is normalized (default is True).
         
        If 'sample_count' is not None, the true combination is approximated using the specified number of samples.
        In this case, 'importance_sampling' determines the method of approximation (only if normalization=True, otherwise 'importance_sampling' is ignored).
        The default method (importance_sampling=False) independently generates samples from both mass functions and computes their intersections.
        If importance_sampling=True, importance sampling is used to avoid empty intersections, which leads to a lower approximation error but is also slower.
        This method should be used if there is significant evidential conflict between the mass functions.
        """
        return self._combine(mass_function, rule=lambda s1, s2: s1 & s2, normalization=normalization, sample_count=sample_count, importance_sampling=importance_sampling)
    
    def combine_disjunctive(self, mass_function, sample_count=None):
        """
        Disjunctively combines the mass function with another mass function and returns the combination as a new mass function.
        
        The other mass function is assumed to be defined over the same frame of discernment.
        If 'mass_function' is not of type MassFunction, it is assumed to be an iterable containing multiple mass functions that are iteratively combined.
        
        If 'sample_count' is not None, the true combination is approximated using the specified number of samples.
        """
        return self._combine(mass_function, rule=lambda s1, s2: s1 | s2, normalization=False, sample_count=sample_count, importance_sampling=False)
    
    def combine_cautious(self, mass_function):
        """
        Combines the mass function with another mass function using the cautious rule and returns the combination as a new mass function.
        
        For more details, see:
        T. Denoeux (2008), "Conjunctive and disjunctive combination of belief functions induced by nondistinct bodies of evidence",
        Artificial Intelligence 172, 234-264.
        """
        w1 = self.weight_function()
        w2 = mass_function.weight_function()
        w_min = {h:min(w1[h], w2[h]) for h in w1}
        theta = self.frame()
        m = MassFunction({theta:1.0})
        for h, w in w_min.items():
            m_simple = MassFunction({theta:w, h:1.0 - w})
            m = m.combine_conjunctive(m_simple, normalization=False)
        return m
    
    def _combine(self, mass_function, rule, normalization, sample_count, importance_sampling):
        """Helper method for combining two or more mass functions."""
        combined = self
        if isinstance(mass_function, MassFunction):
            mass_function = [mass_function] # wrap single mass function
        for m in mass_function:
            if not isinstance(m, MassFunction):
                raise TypeError("expected type MassFunction but got %s; make sure to use keyword arguments for anything other than mass functions" % type(m))
            if sample_count == None:
                combined = combined._combine_deterministic(m, rule)
            else:
                if importance_sampling and normalization:
                    combined = combined._combine_importance_sampling(m, sample_count)
                else:
                    combined = combined._combine_direct_sampling(m, rule, sample_count)
        if normalization:
            return combined.normalize()
        else:
            return combined
    
    def _combine_deterministic(self, mass_function, rule):
        """Helper method for deterministically combining two mass functions."""
        combined = MassFunction()
        for (h1, v1) in self.items():
            for (h2, v2) in mass_function.items():
                combined[rule(h1, h2)] += v1 * v2
        return combined
    
    def _combine_direct_sampling(self, mass_function, rule, sample_count):
        """Helper method for approximatively combining two mass functions using direct sampling."""
        combined = MassFunction()
        samples1 = self.sample(sample_count)
        samples2 = mass_function.sample(sample_count)
        for i in range(sample_count):
            combined[rule(samples1[i], samples2[i])] += 1.0 / sample_count
        return combined
    
    def _combine_importance_sampling(self, mass_function, sample_count):
        """Helper method for approximatively combining two mass functions using importance sampling."""
        combined = MassFunction()
        for (s1, n) in self.sample(sample_count, as_dict=True).items():
            weight = mass_function.pl(s1)
            for s2 in mass_function.condition(s1).sample(n):
                combined[s2] += weight
        return combined
    
    def combine_gbt(self, likelihoods, normalization=True, sample_count=None, importance_sampling=True):
        """
        Conjunctively combines the mass function with a mass function obtained from a sequence of
        likelihoods via the generalized Bayesian theorem and returns the combination as a new mass function.
        
        Equivalent to 'combine_conjunctive(MassFunction.gbt(likelihoods))'.
        By ignoring incompatible likelihoods, it is generally faster than the former
        method and yields a better Monte-Carlo approximation in case of normalization.
        
        'likelihoods' specifies the conditional plausibilities for a set of singleton hypotheses.
        It can either be a dictionary mapping singleton hypotheses to plausibilities or an iterable
        containing tuples consisting of a singleton hypothesis and a corresponding plausibility value.
        
        All arguments except for 'likelihoods' must be specified as keyword arguments.
        'normalization' determines whether the resulting mass function is normalized, i.e., whether m({}) == 0.
        If 'sample_count' is not None, the true mass function is approximated using the specified number of samples.
        See 'combine_conjunctive' for details on the effect of setting 'importance_sampling'.
        """
        core = self.core() # restrict to generally compatible likelihoods
        if isinstance(likelihoods, dict):
            likelihoods = list(likelihoods.items())
        likelihoods = [l for l in likelihoods if l[1] > 0 and l[0] in core]
        if sample_count == None: # deterministic
            return self.combine_conjunctive(MassFunction.gbt(likelihoods), normalization=normalization)
        else: # Monte-Carlo
            if not normalization: # only use importance sampling in case of normalization
                importance_sampling = False
            combined = MassFunction()
            for s, n in self.sample(sample_count, as_dict=True).items():
                if importance_sampling:
                    compatible_likelihoods = [l for l in likelihoods if l[0] in s]
                    weight = 1.0 - reduce(mul, [1.0 - l[1] for l in compatible_likelihoods], 1.0)
                else:
                    compatible_likelihoods = likelihoods
                if not compatible_likelihoods:
                    continue
                if normalization:
                    empty_mass = reduce(mul, [1.0 - l[1] for l in compatible_likelihoods], 1.0)
                for _ in range(n):
                    rv = [random() for _ in range(len(compatible_likelihoods))]
                    subtree_mass = 1.0
                    hyp = set()
                    for k in range(len(compatible_likelihoods)):
                        l = compatible_likelihoods[k][1]
                        norm = 1.0 if hyp or not normalization else 1.0 - empty_mass / subtree_mass
                        if l / norm > rv[k]:
                            hyp.add(compatible_likelihoods[k][0])
                        else:
                            subtree_mass *= 1.0 - l   # only relevant for negative case
                    if importance_sampling:
                        combined[hyp] += weight
                    else:
                        combined[hyp & s] += 1.0
            if normalization:
                return combined.normalize()
            else:
                return combined
    
    def condition(self, hypothesis, normalization=True):
        """
        Conditions the mass function with 'hypothesis'.
        
        'normalization' determines whether the resulting conjunctive combination is normalized (must be specified as a keyword argument).
        
        Shorthand for self.combine_conjunctive(MassFunction({hypothesis:1.0}), normalization).
        """
        m = MassFunction({MassFunction._convert(hypothesis):1.0})
        return self.combine_conjunctive(m, normalization=normalization)
    
    def conflict(self, mass_function, sample_count=None):
        """
        Calculates the weight of conflict between two or more mass functions.
        
        If 'mass_function' is not of type MassFunction, it is assumed to be an iterable containing multiple mass functions.
        
        The weight of conflict is computed as the (natural) logarithm of the normalization constant in Dempster's rule of combination.
        Returns infinity in case the mass functions are flatly contradicting.
        """
        # compute full conjunctive combination (could be more efficient)
        m = self.combine_conjunctive(mass_function, normalization=False, sample_count=sample_count)
        empty = m[frozenset()]
        m_sum = fsum(m.values())
        diff = m_sum - empty
        if diff == 0.0:
            return float('inf')
        else:
            return -log(diff)
    
    def normalize(self):
        """
        Normalizes the mass function in-place.
        
        Sets the mass value of the empty set to 0 and scales all other values such that their sum equals 1.
        For convenience, the method returns 'self'.
        """
        if frozenset() in self:
            del self[frozenset()]
        mass_sum = fsum(self.values())
        if mass_sum != 1.0:
            for (h, v) in self.items():
                self[h] = v / mass_sum
        return self
    
    def prune(self):
        """
        Removes all non-focal (0 mass) hypotheses in-place.
        
        For convenience, the method returns 'self'.
        """ 
        remove = [h for (h, v) in self.items() if v == 0.0]
        for h in remove:
            del self[h]
        return self
    
    def markov(self, transition_model, sample_count=None):
        """
        Computes the mass function induced by a prior belief (self) and a transition model.
        
        The transition model expresses a joint belief over the frame of this mass function and a new frame.
        The belief over the frame of this mass function is implicitly assumed to be vacuous.
        The transition model is a function returning the conditional belief over the new frame (as a mass function
        if sample_count=None) while taking a singleton hypothesis of the current frame as input.
        The disjunctive rule of combination is then used to construct the mass function over the new frame.
        
        If 'sample_count' is not None, the true mass function is approximated using the specified number of samples.
        In this case, 'transition_model' is expected to take a second argument stating how many samples from the corresponding conditional mass function should be returned.
        The return value in this case is expected to be an iterable over sampled hypotheses from the new frame.
         
        This method can be used to implement the prediction step for estimation in a hidden Markov process (hence the name).
        Under this interpretation, the transition model expresses the mass distribution over successor states given the current state.
        """
        updated = MassFunction()
        if sample_count == None: # deterministic
            for k, v in self.items():
                predicted = None
                for e in k:
                    if predicted == None:
                        predicted = transition_model(e)
                    else:
                        predicted |= transition_model(e)
                for kp, vp in predicted.items():
                    updated[kp] += v * vp
        else: # Monte-Carlo
            for s, n in self.sample(sample_count, as_dict=True).items():
                unions = [[] for _ in range(n)]
                for e in s:
                    ts = transition_model(e, n)
                    for i, t in enumerate(ts):
                        unions[i].extend(t)
                for u in unions:
                    updated[u] += 1.0 / sample_count
        return updated
    
    def map(self, function):
        """
        Maps each hypothesis to a new hypothesis using 'function' and returns the new mass function.
        
        'function' is a function taking a hypothesis as its only input and returning a new hypothesis
        (i.e., a sequence that can be converted to a 'frozenset').
        
        Here are some example use cases:
        
        1. Vacuous extension to a multi-dimensional frame of discernment (m is defined over
        the frame A while the new mass function is defined over the Cartesian product AxB):
            
            B = {'x', 'y', 'z'}
            m.map(lambda h: itertools.product(h, B))
        
        2. Projection to a lower dimensional frame (m is defined over AxBxC such that each hypothesis is
        a set of tuples where each tuple consists of 3 elements; the new mass function is defined over BxC):
        
            m.map(lambda h: (t[1:] for t in h))
        """
        m = MassFunction()
        for (h, v) in self.items():
            m[self._convert(function(h))] += v
        return m
    
    def pignistic(self):
        """Computes the pignistic transformation and returns it as a new mass function consisting only of singletons."""
        p = MassFunction()
        for (h, v) in self.items():
            if v > 0.0:
                size = float(len(h))
                for s in h:
                    p[(s,)] += v / size
        return p.normalize()
    
    def local_conflict(self):
        """
        Computes the local conflict measure.
        
        For more information, see Pal et al. 1993. Uncertainty measures for evidential reasoning II:
        A new measure of total uncertainty. International Journal of Approximate Reasoning.
        
        Only works for normalized mass functions.
        If the mass function is unnormalized, the method returns float('nan')
        
        In case the mass function is a probability function (containing only singleton hypotheses),
        it reduces to the classical entropy measure.
        """
        if self[frozenset()] > 0.0:
            return float('nan')
        c = 0.0
        for (h, v) in self.items():
            if v > 0.0:
                c += v * log(len(h) / v, 2)
        return c
    
    def hartley_measure(self):
        """
        Computes the Hartley-like measure in order to quantify the amount of imprecision.
        
        For more information, see:
        G. J. Klir (1999), "Uncertainty and information measures for imprecise probabilities: An overview",
        International Symposium on Imprecise Probabilities and Their Applications.
        """
        return fsum([v * log(len(h), 2) for h, v in self.items()])
    
    def norm(self, m, p=2):
        """
        Computes the p-norm between two mass functions (default is p=2).
        
        Both mass functions are treated as vectors of mass values.
        """
        d = fsum([(v - m[h])**p for (h, v) in self.items()])
        for (h, v) in m.items():
            if h not in self:
                d += v**p
        return d**(1.0 / p)
    
    def is_compatible(self, m):
        """
        Checks whether another mass function is compatible with this one.
        
        Compatibility means that the mass value of each hypothesis in 'm' is less than
        or equal to the corresponding plausibility given by this mass function.
        """
        return all([self.pl(h) >= v for (h, v) in m.items()])
    
    def sample(self, n, quantization=True, as_dict=False):
        """
        Returns n random samples from the mass distribution.
        
        Hypotheses are drawn with a probability proportional to their mass values (with replacement).
         
        If 'quantization' is True (default), the method performs a quantization of the mass values.
        This means the frequency of a hypothesis h in the sample set is at least int(self[h] * n / t) where t is the sum of all mass values.
        The remaining sample slots (if any) are filled up according to the remainders of the fractions computed in the first step.
        
        The parameter 'as_dict' determines the type of the returned value.
        If 'as_dict' is False (default), a list of length n is returned.
        Otherwise, the result is a dictionary specifying the number of samples for each hypothesis.
        """
        if not isinstance(n, int):
            raise TypeError("n must be int")
        samples = {h:0 for h in self} if as_dict else []
        mass_sum = fsum(self.values())
        if quantization:
            remainders = []
            remaining_sample_count = n
            for (h, v) in self.items():
                fraction = n * v / mass_sum
                quotient = int(fraction)
                if quotient > 0:
                    if as_dict:
                        samples[h] = quotient
                    else:
                        samples.extend([h] * quotient)
                remainders.append((h, fraction - quotient))
                remaining_sample_count -= quotient
            remainders.sort(reverse=True, key=lambda hv: hv[1])
            for h, _ in remainders[:remaining_sample_count]:
                if as_dict:
                    samples[h] += 1
                else:
                    samples.append(h)
        else:
            rv = [uniform(0.0, mass_sum) for _ in range(n)]
            hypotheses = sorted(self.items(), reverse=True, key=lambda hv: hv[1])
            for i in range(n):
                mass = 0.0
                for (h, v) in hypotheses:
                    mass += v
                    if mass >= rv[i]:
                        if as_dict:
                            samples[h] += 1
                        else:
                            samples.append(h)
                        break
        if not as_dict:
            shuffle(samples)
        return samples
    
    def is_probabilistic(self):
        """
        Checks whether the mass function is a probability function.
        
        Returns True if and only if all hypotheses are singletons (normalization is ignored). 
        """
        return all([len(h) == 1 for h in self.keys()])
    
    def sample_probability_distributions(self, n):
        """
        Randomly generates n compatible probability distributions from the mass function.
        
        The result is a list of n independently sampled probability distributions expressed as mass functions.
        This can be useful for estimating various statistical measures like the minimum or maximum entropy consistent with the mass distribution.
        """
        samples = [MassFunction() for _ in range(n)]
        for i in range(n):
            for (h, v) in self.items():
                if len(h) == 1:
                    samples[i][h] += v
                else:
                    rv = [random() for _ in range(len(h))]
                    total = fsum(rv)
                    for k, s in enumerate(h):
                        samples[i][{s}] += rv[k] * v / total
        return samples
    
    def max_bel(self):
        """
        Returns the singleton with the highest belief.
        
        In case there are multiple singletons with maximum belief, only one of them is returned.
        Returns None, if the mass function does not contain any hypotheses.
        """
        return self._max_singleton(self.bel)
    
    def max_pl(self):
        """
        Returns the singleton with the highest plausibility.
        
        In case there are multiple singletons with maximum plausibility, only one of them is returned.
        Returns None, if the mass function does not contain any hypotheses.
        """
        return self._max_singleton(self.pl)
    
    def _max_singleton(self, f):
        st = self.singletons()
        if st:
            value_list = [(f(s), s) for s in st]
            shuffle(value_list)
            return max(value_list)[1]
        else:
            return None
    
    def to_dict(self):
        """Convert a mass function only consisting of singletons to a dictionary by removing each enclosing frozenset."""
        if not self.is_probabilistic():
            raise Exception('mass function must only contain singletons')
        return {tuple(h)[0]:v for h, v in self.items()}
    
    @staticmethod
    def from_dict(d):
        """Convert a dictionary to a mass function by enclosing each key with a frozenset."""
        if isinstance(d, MassFunction):
            return d
        else:
            return MassFunction({frozenset((h,)):v for h, v in d.items()})
    
    @staticmethod
    def from_possibility(poss):
        """
        Constructs a consonant mass function from a possibility distribution.
                
        For more information, see:
        D. Dubois, H. Prade (1982), "On several representations of an uncertainty body of evidence",
        Fuzzy Information and Decision Processes, 167-181.
        """
        if isinstance(poss, MassFunction):
            poss = poss.to_dict() # remove enclosing sets
        H, P = zip(*sorted(poss.items(), key=lambda e: e[1], reverse=True)) # sort possibility values in descending order
        m = MassFunction()
        m[H] = P[-1]
        for i in range(len(H) - 1):
            m[H[:i + 1]] = P[i] - P[i + 1]
        return m
    
    @staticmethod
    def pignistic_inverse(p):
        """
        Constructs a consonant mass function from a pignistic probability distribution by applying the inverse pignistic transformation.
        
        For more information, see:
        D. Dubois, H. Prade, P. Smets (2008), "A definition of subjective possibility",
        International Journal of Approximate Reasoning 48 (2), 352-364.
        """
        p = MassFunction.from_dict(p)
        poss = MassFunction({h1:fsum([min(p[h1], p[h2]) for h2 in p.keys()]) for h1 in p.keys()})
        return MassFunction.from_possibility(poss)
    
    @staticmethod
    def _to_array_index(hypothesis, frame):
        """Map a hypothesis to an array index given a frame of discernment."""
        index = 0
        for i, s in enumerate(frame):
            if s in hypothesis:
                index += 2**i
        return index
    
    @staticmethod
    def _from_array_index(index, frame):
        """Map an array index to a hypothesis given a frame of discernment."""
        hypothesis = set()
        for i, s in enumerate(frame):
            if 2**i & index:
                hypothesis.add(s)
        return frozenset(hypothesis)
    
    def to_array(self, frame):
        """
        Convert the mass function to a NumPy array.
        
        Hypotheses are mapped to array indices using '_to_array_index'.
        The resulting array has 2^n entries where n is the size of the frame of discernment.
        """
        a = numpy.zeros(2**len(frame))
        for h, v in self.items():
            a[MassFunction._to_array_index(h, frame)] = v
        return a
    
    @staticmethod
    def from_array(a, frame):
        """
        Convert a NumPy array to a mass function given a frame of discernment.
        
        Array indices are mapped to hypotheses using '_from_array_index'.
        """
        m = MassFunction()
        for i, v in enumerate(a):
            if v > 0.0:
                m[MassFunction._from_array_index(i, frame)] = v
        return m
    
    @staticmethod
    def _confidence_intervals(histogram, alpha):
        """Compute Goodman confidence intervals."""
        p_lower = {}
        p_upper = {}
        a = chi2.ppf(1. - alpha / len(histogram), 1)
        n = float(sum(histogram.values()))
        for h, n_h in histogram.items():
            delta_h = a * (a + 4. * n_h * (n - n_h) / n)
            p_lower[h] = (a + 2. * n_h - sqrt(delta_h)) / (2. * (n + a))
            p_upper[h] = (a + 2. * n_h + sqrt(delta_h)) / (2. * (n + a))
        return p_lower, p_upper
    
    @staticmethod
    def from_samples(histogram, method='idm', alpha=0.05, s=1.0):
        """
        Generate a mass function from an empirical probability distribution that was obtained from a limited number of samples.
        This makes the expected deviation of the empirical distribution from the true distribution explicit.
        
        'histogram' represents the empirical distribution. It is a dictionary mapping each possible event to the respective
        number of observations (represented as integers).
        
        'method' determines the algorithm used for generating the mass function.
        Except for method 'bayesian', all algorithms are based on the idea that the true probabilities lie within confidence intervals
        represented by the mass function with confidence level 1 - 'alpha'.
        
        The following modes are supported:
        
        'idm': Imprecise Dirichlet model. A small amount of mass (controlled by 's') is assigned to the entire frame.
        For more information on 'idm', see:
        P. Walley (1996), "Inferences from multinomial data: learning about a bag of marbles",
        Journal of the Royal Statistical Society. Series B (Methodological), 3-57.
        
        'maxbel': Maximize the total belief by solving a linear program. (Attention: this becomes very computationally expensive
        for larger numbers of events.)
        
        'maxbel-ordered': Similar to 'maxbel' except that the events are assumed to have a natural order (e.g., intervals), in which case
        the mass function can be computed analytically and thus much faster.
        
        For more information on 'maxbel' and 'maxbel-ordered', see:
        T. Denoeux (2006), "Constructing belief functions from sample data using multinomial confidence regions",
        International Journal of Approximate Reasoning 42, 228-252.
        
        'mcd': Compute the least committed consonant mass function whose pignistic transformation lies within the confidence interval
        induced by 'alpha'. Like 'maxbel', it is based on solving a linear program and quickly becomes computationally expensive.
        
        'mcd-approximate': An approximation of 'mcd' that can be computed much more efficiently.
        
        For more information on these two methods, see:
        A. Aregui, T. Denoeux (2008), "Constructing consonant belief functions from sample data using confidence sets of pignistic probabilities",
        International Journal of Approximate Reasoning 49, 575-594.
        
        'bayesian': Construct a Bayesian mass function based on the relative frequencies. In addition, additive smoothing is applied (controlled by 's'). 
        
        In case the sample number is 0, returns a vacuous mass function (or uniform distribution for 'bayesian').
        
        (Requires SciPy for computing confidence intervals and solving linear programs.)
        """
        if not isinstance(histogram, dict):
            raise TypeError('histogram must be of type dict')
        for v in histogram.values():
            if not isinstance(v, int):
                raise TypeError('all histogram values must be of type int')
        if not histogram:
            return MassFunction()
        if sum(histogram.values()) == 0: # return vacuous/uniform belief if there are no samples
            vac = MassFunction({tuple(histogram.keys()):1})
            if method == 'bayesian':
                return vac.pignistic()
            else:
                return vac
        if method == 'bayesian':
            return MassFunction({(h,):v + s for h, v in histogram.items()}).normalize()
        elif method == 'idm':
            return MassFunction._from_samples_idm(histogram, s)
        elif method == 'maxbel':
            return MassFunction._from_samples_maxbel(histogram, alpha)
        elif method == 'maxbel-ordered':
            return MassFunction._from_samples_maxbel(histogram, alpha, ordered=True)
        elif method == 'mcd':
            return MassFunction._from_samples_mcd(histogram, alpha)
        elif method == 'mcd-approximate':
            return MassFunction._from_samples_mcd(histogram, alpha, approximate=True)
        raise ValueError('unknown method: %s' % method)
    
    @staticmethod
    def _from_samples_idm(histogram, s):
        """
        Reference:
        P. Walley (1996), "Inferences from multinomial data: learning about a bag of marbles",
        Journal of the Royal Statistical Society. Series B (Methodological), 3-57.
        """
        total = sum(histogram.values())
        m = MassFunction()
        for h, c in histogram.items():
            m[(h,)] = float(c) / (total + s)
        m[MassFunction._convert(histogram.keys())] = float(s) / (total + s)
        return m
    
    @staticmethod
    def _from_samples_maxbel(histogram, alpha, ordered=False):
        """
        Reference:
        T. Denoeux (2006), "Constructing belief functions from sample data using multinomial confidence regions",
        International Journal of Approximate Reasoning 42, 228-252.
        """
        p_lower, p_upper = MassFunction._confidence_intervals(histogram, alpha)
        def p_lower_set(hs):
            l = u = 0
            for h in H:
                if h in hs:
                    l += p_lower[h]
                else:
                    u += p_upper[h]
            return max(l, 1 - u)
        if ordered:
            H = sorted(histogram.keys())
            m = MassFunction()
            for i1, h1 in enumerate(H):
                m[(h1,)] = p_lower[h1]
                for i2, h2 in enumerate(H[i1 + 1:]):
                    i2 += i1 + 1
                    if i2 == i1 + 1:
                        v = p_lower_set(H[i1:i2 + 1]) - p_lower[h1] - p_lower[h2]
                    else:
                        v = p_lower_set(H[i1:i2 + 1]) - p_lower_set(H[i1 + 1:i2 + 1]) - p_lower_set(H[i1:i2]) + p_lower_set(H[i1 + 1:i2])
                    if v > 0:
                        m[H[i1:i2 + 1]] = v
            return m
        else:
            H = list(histogram.keys())
            L = 2**len(H)
            initial = numpy.zeros(L)
            cons = []
            singletons = lambda index: [i for i in range(len(H)) if 2**i & index]
            # constraint (24)
            bel = lambda index, m: fsum(m[sum([2**i for i in h_ind])] for h_ind in powerset(singletons(index)))
            c24 = lambda m, i: p_lower_set(MassFunction._from_array_index(i, H)) - bel(i, m)
            for i in range(L):
                cons.append(partial(c24, i=i))
            # constraint (25)
            cons.append(lambda m: m.sum() - 1.0)
            cons.append(lambda m: 1.0 - m.sum())
            # constraint (26)
            for i in range(L):
                cons.append(partial(lambda m, i_s: m[i_s], i_s=i))
            f = lambda m: -1 * 2**len(H) * fsum([m[i] * 2**(-len(singletons(i))) for i in range(L)])
            m_optimal = fmin_cobyla(f, initial, cons, disp=0)
            return MassFunction.from_array(m_optimal, H)
        
    @staticmethod
    def _from_samples_mcd(histogram, alpha, approximate=False):
        """
        Reference:
        A. Aregui, T. Denoeux (2008), "Constructing consonant belief functions from sample data using confidence
        sets of pignistic probabilities", International Journal of Approximate Reasoning 49, 575-594.
        """
        p_lower, p_upper = MassFunction._confidence_intervals(histogram, alpha)
        H = list(histogram.keys())
        if approximate:
            # approximate possibility distribution
            poss = {h1:min(1, fsum([min(p_upper[h1], p_upper[h2]) for h2 in H])) for h1 in H}
        else:
            # optimal possibility distribution (based on linear programming)
            poss = {h:0. for h in H}
            for k, h_k in enumerate(H):
                S_k = {l for l in range(len(H)) if p_lower[H[l]] >= p_upper[h_k]}
                S_k.add(k)
                I_k = {l for l in range(len(H)) if p_upper[H[l]] < p_lower[h_k]}
                P_k = set(range(len(H))).difference(S_k.union(I_k))
                for A in powerset(P_k):
                    G = S_k.union(A)
                    G_c = set(range(len(H))).difference(G)
                    cons = []
                    # constraint (26)
                    for i, h in enumerate(H):
                        cons.append(partial(lambda p, i_s, p_s: p[i_s] - p_s, i_s=i, p_s=p_lower[h])) # lower bound
                        cons.append(partial(lambda p, i_s, p_s: p_s - p[i_s], i_s=i, p_s=p_upper[h])) # upper bound
                    # constraint (27)
                    cons.append(lambda p: 1. - sum(p))
                    cons.append(lambda p: sum(p) - 1.)
                    # constraint (30)
                    for i in G:
                        cons.append(partial(lambda p, i_s: p[i_s] - p[k], i_s=i))
                    # constraint (31)
                    for i in G_c:
                        cons.append(partial(lambda p, i_s: p[k] - p[i_s], i_s=i))
                    initial = [1.0 / len(H)] * len(H)
                    f = lambda p: -(fsum([p[i] for i in G_c]) + len(G) * p[k])
                    poss_optimal = fmin_cobyla(f, initial, cons, disp=0)
                    poss[h_k] = max(poss[h_k], -f(poss_optimal))
        return MassFunction.from_possibility(poss)


def powerset(iterable):
    """
    Returns an iterator over the power set of 'set'.
    
    'set' is an arbitrary iterator over hashable elements.
    All returned subsets are of type 'frozenset'.
    """
    return map(frozenset, chain.from_iterable(combinations(iterable, r) for r in range(len(iterable) + 1)))

def gbt_m(hypothesis, likelihoods, normalization=True):
    """
    Computes the mass value of 'hypothesis' using the generalized Bayesian theorem.
    
    Equivalent to MassFunction.gbt(likelihoods, normalization)[hypothesis].
    """
    if isinstance(likelihoods, dict):
        likelihoods = list(likelihoods.items())
    q = gbt_q(hypothesis, likelihoods, normalization)
    return q * reduce(mul, [1.0 - l[1] for l in likelihoods if l[0] not in hypothesis], 1.0)

def gbt_bel(hypothesis, likelihoods, normalization=True):
    """
    Computes the belief of 'hypothesis' using the generalized Bayesian theorem.
    
    Equivalent to MassFunction.gbt(likelihoods, normalization).bel(hypothesis).
    """
    if isinstance(likelihoods, dict):
        likelihoods = list(likelihoods.items())
    eta = _gbt_normalization(likelihoods) if normalization else 1.0
    exc = reduce(mul, [1.0 - l[1] for l in likelihoods if l[0] not in hypothesis], 1.0)
    all_hyp = reduce(mul, [1.0 - l[1] for l in likelihoods], 1.0)
    return eta * (exc - all_hyp)

def gbt_pl(hypothesis, likelihoods, normalization=True):
    """
    Computes the plausibility of 'hypothesis' using the generalized Bayesian theorem.
    
    Equivalent to MassFunction.gbt(likelihoods, normalization).pl(hypothesis).
    """
    if isinstance(likelihoods, dict):
        likelihoods = list(likelihoods.items())
    eta = _gbt_normalization(likelihoods) if normalization else 1.0
    return eta * (1.0 - reduce(mul, [1.0 - l[1] for l in likelihoods if l[0] in hypothesis], 1.0))
    
def gbt_q(hypothesis, likelihoods, normalization=True):
    """
    Computes the commonality of 'hypothesis' using the generalized Bayesian theorem.
    
    Equivalent to MassFunction.gbt(likelihoods, normalization).q(hypothesis).
    """
    if isinstance(likelihoods, dict):
        likelihoods = list(likelihoods.items())
    eta = _gbt_normalization(likelihoods) if normalization else 1.0
    return eta * reduce(mul, [l[1] for l in likelihoods if l[0] in hypothesis], 1.0)

def gbt_pignistic(singleton, likelihoods):
    """
    Computes the pignistic probability of 'singleton' for the belief function obtained
    by applying the generalized Bayesian theorem to 'likelihoods'.
    
    This function has time complexity O(len(likelihoods)**2) and is equivalent to the following
    expression (which has exponential complexity):
    MassFunction.gbt(likelihoods).pignistic()[(singleton,)]
    """
    if isinstance(likelihoods, dict):
        likelihoods = list(likelihoods.items())
    singleton_lh = None
    lh_values = []
    for h, v in likelihoods:
        if h == singleton:
            singleton_lh = v
        else:
            lh_values.append(v)
    if singleton_lh is None:
        raise ValueError('singleton %s is not contained in likelihoods' % repr(singleton))
    m_sum = _gbt_pignistic_recursive(lh_values, 0)
    eta = _gbt_normalization(likelihoods)
    return sum([eta * v * singleton_lh / (c + 1.) for c, v in enumerate(m_sum)])

def _gbt_pignistic_recursive(likelihoods, i):
    """
    Helper function for recursively computing the pignistic probability corresponding to the GBT.
    
    This function computes the sum over all mass values (obtained via the GBT) and groups them by the
    cardinalities of the corresponding sets.
    """
    if i == len(likelihoods) - 1:
        m_sum = [0.] * (len(likelihoods) + 1)
        m_sum[0] = 1. - likelihoods[i]
        m_sum[1] = likelihoods[i]
        return m_sum
    else:
        m_sum = _gbt_pignistic_recursive(likelihoods, i + 1)
        m_sum_inc = [0.] * (len(likelihoods) + 1)
        m_sum_exc = [0.] * (len(likelihoods) + 1)
        for k in range(len(likelihoods) + 1):
            if k < len(likelihoods):
                m_sum_inc[k+1] = m_sum[k] * likelihoods[i]
            m_sum_exc[k] = m_sum[k] * (1. - likelihoods[i])
            m_sum[k] = m_sum_inc[k] + m_sum_exc[k]
        return m_sum

def _gbt_normalization(likelihoods):
    """Helper function for computing the GBT normalization constant."""
    return 1.0 / (1.0 - reduce(mul, [1.0 - l[1] for l in likelihoods], 1.0))


# Naive Adaptive Sensor Fusion

In [2]:
# -*- coding: utf-8 -*-
"""
Created on Tue Oct 29 18:13:08 2019

@author: 14342
"""
from pyds_local import *

import random
import numpy as np
import sys
import pandas as pd
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.base import clone
import itertools
import sys
import re

class DS_Model:
    
    def __init__(self, resp_var_1, resp_var_2, X_train, y_train, feature_set_idx):
        """
        Initializing a DS model

        Parameters
        ----------
        resp_var_1 : list
        resp_var_2 : list
            These two are the response variables that we want to build a model
            on. For example if we are building a model to classify {c1} vs {c2,c3}
            then [c1] would be our resp_var_1 and [c2,c3] would be our resp_var_2.
            
        X_train : pd.dataframe
            training features in a dataframe format.
        y_train : pd.dataframe
            training labels as a vector.
        feature_set_idx : integer
            Index of the feature set (out of all the randomly created feature sets).

        Returns
        -------
        None.

        """
        self.Feature_Set_idx = feature_set_idx
        self.Response_Variables = [resp_var_1,resp_var_2]
        self.clf = LogisticRegression(class_weight='balanced',solver='saga', n_jobs=-1).fit(X_train, y_train)
        self.Bags = []                          #list of the bags for this model
        self.Uncertainty_B = 0                  #Uncertainty of the biased model
        self.mass = MassFunction()              #Mass function of the model
    
    def Mass_Function_Setter(self, uncertainty, X):
        """
        We used pyds package (a dempster shafer package) to define the mass function
        given the probabilities and uncertainty.
        """
        probability = self.clf.predict_proba(X)
        self.mass[self.Response_Variables[0]] = probability[0,0]*(1-uncertainty)
        self.mass[self.Response_Variables[1]] = probability[0,1]*(1-uncertainty)
    
    def Bags_Trainer(self, X_train, y_train, ratio, num_bags):
        """
        This function trains bags

        Parameters
        ----------
        X_train : pd.dataframe
            training features in a dataframe format.
        y_train : pd.dataframe
            training labels as a vector.
        ratio : float
            Ratio of the generated bagging size to the actual training dataset.
        num_bags : integer
            number of the bags to generate.

        Returns
        -------
        None.

        """
        for i in range(num_bags):
            self.Bags.append(clone(self.clf))
            indices = random.choices(list(range(len(X_train))), k=int(ratio*(len(X_train))))
            X_train_Bag = X_train.iloc[indices,:]
            y_train_Bag = y_train.iloc[indices]
            
            self.Bags[i].fit(X_train_Bag, y_train_Bag)
    
    def Uncertainty_Context(self, X_test_single):
        """
        This function calculates the uncertainty of the contextual meaning by
        calculating the votes from all the bags.

        Parameters
        ----------
        X_test_single : pd.series
            one single test example.

        Returns
        -------
        None.

        """
        
        C = len(self.Response_Variables) #Number of the classes
        V = np.zeros(C) #Vote vector
        T = len(self.Bags) #total number of the bags
        
        for i in range(T):
            V[int(self.Bags[i].predict(X_test_single))] += 1
        
        Uncertainty_c = 1 - np.sqrt(np.sum(np.power((V/T-1/C),2)))/np.sqrt((C-1)/C)
        return(Uncertainty_c)
        
        
        

class ProgressBar(object):
    DEFAULT = 'Progress: %(bar)s %(percent)3d%%'
    FULL = '%(bar)s %(current)d/%(total)d (%(percent)3d%%) %(remaining)d remaining feature sets'

    def __init__(self, total, width=40, fmt=DEFAULT, symbol='=',
                 output=sys.stderr):
        assert len(symbol) == 1

        self.total = total
        self.width = width
        self.symbol = symbol
        self.output = output
        self.fmt = re.sub(r'(?P<name>%\(.+?\))d',
            r'\g<name>%dd' % len(str(total)), fmt)

        self.current = 0

    def __call__(self):
        percent = self.current / float(self.total)
        size = int(self.width * percent)
        remaining = self.total - self.current
        bar = '[' + self.symbol * size + ' ' * (self.width - size) + ']'

        args = {
            'total': self.total,
            'bar': bar,
            'current': self.current,
            'percent': percent * 100,
            'remaining': remaining
        }
        print('\r' + self.fmt % args, file=self.output, end='')

    def done(self):
        self.current = self.total
        self()
        print('', file=self.output)


def feature_set(sensors_to_fuse, st_feat, Sensors, feat_set_count=100):
    """
    This function takes a list of name of the sensors to fuse "sensors_to_fuse
    and structure of the reduced feature space also as a list (like [s1,s2,..])
    where "si" is the number of the features of the ith sensor to be used and 
    also number of the reduced feature sets to create. Then it returns a matrix
    with each row being a feature set number of columns
       
    Parameters
    ----------
    sensors_to_fuse: list
        a list of the name of the sensors to fuse. like ['Acc','Gyro','PS','Aud']
    st_feat: list
        the structure of a feature set. like [s1,s2,..] where "si" is 
        the number of the features of the ith sensor
    feat_set_count: integer
        the number of randome feature sets to create. It is a number like 100
    Sensors: dict
        the sensors dictionary which is the output of the labeling function
       
    Returns
    -------
        selected_feats: np.array 2D
            a matrix that each row represents one set of features and the 
            values in each rows are the index of the columns of the data to
            be used as features
       
    """

    #Making sure that the length of the "st_feat" is equal to the length of the
    #"sensors_to_fuse
    assert len(st_feat) == len(sensors_to_fuse)
    
    #Creating and initializing the "selected_feats" to 0
    selected_feats = np.zeros([feat_set_count,sum(st_feat)])
    
    #A for loop to create the desired number of the random reduced feature sets
    for j in range(feat_set_count):
        
        #"col" stores the index of the randomly generated columns (features)
        col = []
        for i in range(len(st_feat)):
            
            #"aux" stores the index of the generated columns of one sensor
            aux = random.sample(Sensors[sensors_to_fuse[i]],st_feat[i])
            aux.sort()
            col.extend(aux)
        selected_feats[j,:] = col
    return(selected_feats)


def BPA_builder(labels):
    """This function creates the Basic Probability Assignment (BPA) matrix given
       a list of labels (classes)
       
       Input:
           labels: a list of the target labels to be used as our propositins.
                   In other words it is the Frame of discernment (FOD).
                   like: ['Walking','Lying_down']
       Ouput:
           mass: a dataframe that has all the subsets of the the FOD. each row
                 respresent a sebset with a binary vector like:
                     
                     Lying_down     Sleeping    mass
                     0              0           0
                     0              1           0
                     1              0           0
                     1              1           0
                 
                 ex. the last row represents {'Lying_down','Sleeping'} and m is
                 the corresponding basic beleif assignment or mass function.
                 also all masses are initialized with 0
                   
    """
       
    mass = pd.DataFrame(columns=labels)
    perms = list(itertools.product([0, 1], repeat=len(labels)))
    for i in range(len(perms)):
        mass.loc[i] = perms[i]
    mass['m'] = 0
    return(mass)
    
    
def pair_resp(BPA):
    """This function takes in the mss function (dataframe in fact) and returns
       two lists. Each element in each of the lists is itself a list of labels
       representing one subset of the FOD or one member of the power set. Note
       that perms_set_1[i] and perms_set_2[i] are complementary subsets.
       
       Input:
           BPA: the mass dataframe. output of the BPA_builder()
       
       Output:
           perms_set_1: a list of lists
           perms_set_2: the same
           
       ex. target labels = 'Lying_down','Sleeping','Walking'
           perms_set_1[2]=['Lying_down','Sleeping'] 
           perms_set_2[2]=['Walking']
    
    """
    
    perms_set_1 = []
    perms_set_2 = []
    
    #Here we don't need the last column of the mass which is the valuses of mass
    #function. So we get rid of them so that they won't interfere later
    BPA = BPA.iloc[:,:-1]
    
    #It is enough to go up to the half of the rows of the mass to generate the
    #two classes of permustations. The rest are just the complementary to the
    #first half. In fact if the powerset has 2**n elements, we have 2**(n-1) 
    #complementary pairs
    for i in range(BPA.shape[0]//2):
        assert (sum(BPA.loc[i,:])+sum(BPA.loc[BPA.shape[0]-i-1,:])) == BPA.shape[1]
        perms_set_1.append([])
        perms_set_2.append([])
        
        for j in range(BPA.shape[1]):
            if BPA.iloc[i,j] == 1:
                perms_set_1[i].append(BPA.columns[j])
            else:
                perms_set_2[i].append(BPA.columns[j])
    
    perms_set_1.remove(perms_set_1[0])
    perms_set_2.remove(perms_set_2[0])
    
    return(perms_set_1,perms_set_2)


def Uncertainty_Bias(response_variables):
    """
    This function calculates the "uncertainty of the biased model" based on the
    frequency of the classes.
    
    Input:
        response_variables[list of pd.Dataframe]: each element of the list is 
        itself a dataframe of one class (response variable).
    
    Output:
        U[float]: uncertainty of the biased model
    """
    
    C = len(response_variables) #number of the classes (response variables)
    I = np.zeros(C) #an array of the frequencies of the classes
    
    S = 0 #total samples in the dataset
    for i in range(C):
        I[i] = int(np.sum(response_variables[i] == 1))
        S += I[i]
    
    U = np.sqrt(np.sum(np.power((I/S-1/C),2)))/np.sqrt((C-1)/C)
    return (U)

    
def Model_Selector(uncertainty_m, models_per_rp, num_rp, fs_axis):
    """This function takes the total uncertainty matrix and chooses the least
    uncertain models for every response permutation.
    
    Input:
        uncertainty_m[np.array 2d]: Matrix of the total uncertainty for all models.
        models_per_rp[int]: Number of the models to select for each response permutation
        num_rp[int]: Number of the response variables (Also the length of the 
              uncertainty matrix along one of the dimensions)
        fs_axis[int]: The axis of the uncertainty matrix along which feature sets
            are laid
    
    Output:
        selected_models_idx[np.array 2d]: A 2d array that holds the feature set
            indices for each response permutation
    """
    
    selected_models_idx = np.zeros([num_rp, models_per_rp])
    
    index_m = np.argsort(uncertainty_m, axis= fs_axis)
    
    for i in range(models_per_rp):
        selected_models_idx[:,i] = np.argwhere(index_m == i)[:,1]
    
    return(selected_models_idx)
        
def Fuse_and_Predict(selected_models_idx, Models, FOD, num_classes, num_rp, models_per_rp):
    assert len(selected_models_idx) == num_rp
    y_pred = np.zeros([1,num_classes])
    
    fs_idx = int(selected_models_idx[0][0])
    combined_mass = Models[0][fs_idx].mass
    for i in range(num_rp):
        for j in range(models_per_rp):
            if i == 0 and j == 0:
                continue
            else:
                fs_idx = int(selected_models_idx[i][j])
                combined_mass = combined_mass & Models[i][fs_idx].mass
    
    for i in range(num_classes):
        y_pred[0,i] = combined_mass[{FOD[i]}]
    
    y_pred_aux = np.zeros_like(y_pred)
    y_pred_aux[np.arange(len(y_pred)), y_pred.argmax(1)] = 1
    y_pred = y_pred_aux
    
    return(y_pred)
    
# def NAPS_Models_Trainer(num_rp, fs_range, train_dataset, feature_sets, Response_Perm_1, \
#                         Response_Perm_2, impute = True):
#     """
#     This function trains NAPS models.

#     Parameters
#     ----------
#     num_rp : int
#         number of the response permutations (which is 2**(n-1)-1 for n classes).
#     fs_range : range or list
#         a list of int from 0 to m (m being the total number of the feature sets).
#     train_dataset : pd.dataframe
#         dataframe of the training features.
#     feature_sets : np.array 2D
#         matrix of the selected features.
#     Response_Perm_1 : list
#         lsit of the response permutations.
#     Response_Perm_2 : list
#         complementary list of the Response_Perm_1.
#     impute : bool, optional
#         Whether to impute or discard missing features. The default is True.

#     Returns
#     -------
#     None.

#     """
#     NAPS_models = []
#     print('\nLooping over Response Permutations \n ')
    
#     for i in range(num_rp):
#         print('\nResponse Permutation {}/{}'.format(i+1,num_rp))
#         print('\n\tLooping over feature sets')
#         NAPS_models.append([])
                
#         for j in fs_range:
#             print('\t\tFeature Set {}/{}'.format(j+1, len(fs_range)))
#             NAPS_models[i].append([])
#             #find X and y
#             X_train, y_train, y1, y2 = Xy(train_dataset, feature_sets, j, \
#                             Response_Perm_1, Response_Perm_2, i, impute = True)
#     #        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
            
#             X_train_tmp, y_train_tmp = smt.fit_sample(X_train, y_train)
            
#             X_train = pd.DataFrame(X_train_tmp, columns = X_train.columns)
#             y_train = pd.DataFrame(y_train_tmp, columns = ['Merged_label'])
            
            
#             if j == 0:
#                 U2 = Uncertainty_Bias([y1, y2])
    
#             #create a model and train it
#             NAPS_models[i][j] = DS_Model(Response_Perm_1[i], Response_Perm_2[i], \
#                        X_train, y_train, j)
#             NAPS_models[i][j].Bags_Trainer(X_train, y_train, bagging_R, num_bags)
#             NAPS_models[i][j].Uncertainty_B = U2
    
#     return(NAPS_models)
        
    
    
    
    
    



# Config

In [3]:
# -*- coding: utf-8 -*-
"""
Created on Mon Jun 15 21:05:15 2020

@author: 14342
"""
data_dir = "C:\\Users\\mehrd\\OneDrive\\Documents\\GitHub\\NAPS-Fusion\\datasets"
cvdir = "C:\\Users\\mehrd\\OneDrive\\Documents\\GitHub\\NAPS-Fusion\\cv5Folds\\cv_5_folds\\"

sensors_to_fuse = ['Acc','Gyro','W_acc','Aud'] 
#there are also "Mag", "Loc", "AP", "PS", "LF", "Compass" sensors
FOD = ['label:LYING_DOWN','label:SITTING','label:OR_standing','label:FIX_walking']#\
#      ,'label:FIX_running','label:BICYCLING']
feature_sets_st = [3,3,3,3] #feature set structure
feature_sets_count = 10
bagging_R = 0.6  #bagging ratio
num_bags = 4
models_per_rp = 2  #number of models to select for the fusion per response permutation
feature_range = range(1,225) #range of the column number of all features
num_prc = 6  #number of processors to split the job during parallelization



# Main

In [4]:
# -*- coding: utf-8 -*-
"""
Created on Wed Oct 30 13:17:55 2019

@author: 14342
"""
from __future__ import absolute_import, division, print_function
from pyds_local import *
from Naive_Adaptive_Sensor_Fusion import *
from config import *

from timeit import default_timer as timer
import random
import glob
import os
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import warnings
from imblearn.over_sampling import SMOTE
import timeit
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import f1_score
from multiprocessing import Pool, TimeoutError
#import tensorflow as tf

warnings.filterwarnings("ignore")


def readdata_csv(data_dir):
    """This function gets the directory of the datasets and returns the dataset
    containing information of all 60 users
    
    Input:
        data_dir[string]: holds the directory of all the csv files (60)
        
    Output:
        grand_dataset[dict]: a dictionary of all the users' data. The format is:
            grand_dataset:{'uuid1': dataframe of csv file of the user 1
                           'uuid2': dataframe of csv file of the user 2
                           'uuid3': dataframe of csv file of the user 3
                           ...}
    """
    length_uuids = 36 # number of characters for each uuid
    data_list = glob.glob(os.path.join(os.getcwd(), data_dir, "*.csv"))
    # grand_dataset is a dict. that holds the uuids and correspondong datast
    grand_dataset = {}
    for i in range(len(data_list)):
#    for i in range(5):
        # dismantles the file name and picks only uuids (first 36 characters)
        uuid = os.path.basename(data_list[i])[:length_uuids]    
        dataset_ith = pd.read_csv(data_list[i])
        print('User {}/{}  -> Shape of the data     {}'.format(i+1, \
              len(data_list), dataset_ith.shape))
        grand_dataset[uuid] = dataset_ith
    return(grand_dataset)
    
def Set_Act_Sens():
    """This function defines two dictionaries for activities and sensors. Each
    dictionaray holds the the range of columns for the specified sensor or 
    activity.
    
    Input:
    Output:
        Activities[dict]: a dictionary of the activities and their corresponding
                        column number
        Sensors[dict]: a dictionary of the sensors and their corresponding range
                        of features
    """
    Activities = {}
    Activities['label:LYING_DOWN'] = 226
    Activities['label:SITTING'] = 227
    Activities['label:FIX_walking'] = 228
    Activities['label:FIX_running'] = 229
    Activities['label:BICYCLING'] = 230
    Activities['label:SLEEPING'] = 231
    Activities['label:OR_standing'] = 270
    
    
    Sensors = {}
    Sensors['Acc'] = list(range(1,27))
    Sensors['Gyro'] = list(range(27,53))
#    Sensors['Mag'] = list(range(53,84))
    Sensors['W_acc'] = list(range(84,130))
#    Sensors['Compass'] = list(range(130,139))
    Sensors['Loc'] = list(range(139,156))
    Sensors['Aud'] = list(range(156,182))
#    Sensors['AP'] = list(range(182,184))
    Sensors['PS'] = list(np.append(range(184,210),range(218,226)))
#    Sensors['LF'] = list(range(210,218))
    
    return(Activities,Sensors)
    
def Response_Merger(data, cols_to_merge):
    """
    This function takes in the dataset and a list of columns of different labels 
    to merge and combnine them using a logical OR to give back one column. ex. 
    l1+l2+l3 -> {l1,l2,l3}

    Parameters
    ----------
    data : dataframe
        dataframe of the dataset (ex. training data).
    cols_to_merge : list
        a list of the columns to merge with a logical OR. like:
        ['Lying_down','Sleeping'].

    Returns
    -------
    merged_label: dataframe
        a dataframe with only one column whose values are binary

    """
    data = data[cols_to_merge].fillna(0)
    
    merged_label = data[cols_to_merge[0]]
    
    for i in range(1,len(cols_to_merge)):
        merged_label = np.logical_or(merged_label, data[cols_to_merge[i]])*1
    
    merged_label = merged_label.to_frame()
    merged_label.columns=['Merged_label']

     
#    col_name = ''.join(cols_to_merge[:])
#    cols_to_merge = add_label(cols_to_merge)
#        
#    # First we impute the NaN with 0 in all the columns that are about to be merged
#    data = data[cols_to_merge].fillna(0)
#    
#    # Now find the logical OR of the desired columns (labels)
#    merged_label = data[cols_to_merge[0]]
#    merged_label.name = col_name
#    
#    for i in range(len(cols_to_merge)):
#        merged_label = np.logical_or(merged_label, data[cols_to_merge[i]])
    return(merged_label)

def Xy(data, feature_sets, feature_set_idx, response_perms_1, response_perms_2,\
       response_perm_idx, impute = False):
    """This function takes data, feature sets matrix, respnse perms, the index
       of the row of the desired feature set, and the index of the desired rows
       of the response variables and gives back the X and y"""
    # Maybe add sorting later
    # Maybe some more manupulations on response variable
    X = data.iloc[:,list(feature_sets[feature_set_idx,:])]
#    X.loc[:,:] = preprocessing.scale(X)
    
    if impute is not False:
        X = X.fillna(0)
    
    y1 = Response_Merger(data, response_perms_1[response_perm_idx])
    y2 = Response_Merger(data, response_perms_2[response_perm_idx])
    
    aux = y1 + y2
    indices = np.where(aux ==1)[0]
    
    if len(indices) > 1:
        y1 = y1.loc[indices,:]
        y2 = y2.loc[indices,:]
        X = X.loc[indices,:]
    
    y = y1
    
    xy = pd.concat([X,y],axis=1)
    xy = xy.dropna()
    
    y = xy.iloc[:,-1]
    X = xy.iloc[:,0:-1]
    
    return(X, y, y1, y2)

def train_test_spl(test_fold, num_folds, fold_dir, grand_dataset):
    """This function takes the number of test fold (ranging from 0 to 4) and
    number of folds (in this case 5) and directory where the folds' uuids are
    and the dataset, and returns train and test datasets
    
    Input:
        test_fold_idx[integer]: an integer indicating the index of the test fold
        fold_dir[string]: holds the directory in which the folds' uuids are
        grand_dataset[dict]: a dictionary of all users' data. (essentially the
                             output of readdata_csv())
    Output:
        train_dataset[pandas.dataframe]: dataframe of the train dataset
        test_dataset[pandas.dataframe]: dataframe of the test dataset
    """
    train_dataset = pd.DataFrame()
    test_dataset = pd.DataFrame()
    folds_uuids = get_folds_uuids(fold_dir)
    
    # Dividing the folds uuids into train and test (the L denotes they are still lists)
    test_uuids_L = [folds_uuids[test_fold]]
    del(folds_uuids[test_fold])
    train_uuids_L = folds_uuids
    
    # Transforming the list of arrays of uuids into a single uuids np.array
    test_uuids = np.vstack(test_uuids_L)
    train_uuids = np.vstack(train_uuids_L)
    
    # Now collecting the test and train dataset using concatenating
    for i in train_uuids:
        train_dataset = pd.concat([train_dataset,grand_dataset[i[0]]], axis=0, \
                                  ignore_index=True)
    
    for j in test_uuids:
        test_dataset = pd.concat([test_dataset,grand_dataset[j[0]]], axis=0, \
                                 ignore_index=True)
        
    return(train_dataset,test_dataset)

def get_folds_uuids(fold_dir):
    """
    The function gets the directory where the the folds text files are located
    and returns a list of five np.arrays in each of them the uuids of the
    corresponding fold are stored.
    
    Input:
        fold_dir[string]: holds the directory in which folds are
    
    Output:
        folds_uuids[list]: a list of numpy arrays. Each array holds the uuids
                    in that fold. ex.
                    folds_uuids = [('uuid1','uuid2',...,'uuid12'),
                                   ('uuid13','uuid14',...,'uuid24'),
                                   ...,
                                   ('uuid49','uuid50',...,'uuid60')]
    """
    num_folds = 5
    # folds_uuids is gonna be a list of np.arrays. each array is a set of uuids
    folds_uuids = [0,1,2,3,4]
    # This loop reads all 5 test folds (iphone and android) and stores uuids
    for i in range(0,num_folds):
        filename = 'fold_{}_test_android_uuids.txt'.format(i)
        filepath = os.path.join(fold_dir, filename)
        # aux1 is the uuids of ith test fold for "android"
        aux1 = pd.read_csv(filepath,header=None,delimiter='\n')
        aux1 = aux1.values
        
        filename = 'fold_%s_test_iphone_uuids.txt' %i
        filepath = os.path.join(fold_dir, filename)
        # aux2 is the uuids of ith test fold for "iphone"
        aux2 = pd.read_csv(filepath,header=None,delimiter='\n')
        aux2 = aux2.values
        
        # Then we concatenate them
        folds_uuids[i] = np.concatenate((aux1,aux2),axis=0)
        
    return(folds_uuids)

def train_NAPS_Models(train_dataset, feature_sets, j, Response_Perm_1, \
                      Response_Perm_2, i, bagging_R, num_bags, impute = True):
    
    t0 = timeit.default_timer()
    X_train, y_train, y1, y2 = Xy(train_dataset, feature_sets, j, \
                        Response_Perm_1, Response_Perm_2, i, impute = True)
    
    t1 = timeit.default_timer()
    # print('\tgetting training data took : ', t1-t0)
    
    X_train_tmp, y_train_tmp = smt.fit_resample(X_train, y_train)
    t2 = timeit.default_timer()
    # print('\tSMOTE took : ', t2-t1)
    
    X_train = pd.DataFrame(X_train_tmp, columns = X_train.columns)
    y_train = pd.DataFrame(y_train_tmp, columns = ['Merged_label'])
    
    
    U2 = Uncertainty_Bias([y1, y2])
    
    t3 = timeit.default_timer()
    #create a model and train it
    NAPS_sample = DS_Model(Response_Perm_1[i], Response_Perm_2[i], \
               X_train, y_train, j)
    t4 = timeit.default_timer()
    # print('\tCreating the DS model took : ', t4-t3)
    
    NAPS_sample.Bags_Trainer(X_train, y_train, bagging_R, num_bags)
    t5 = timeit.default_timer()
    # print('\tTraining bags took : ', t5-t4)
    
    NAPS_sample.Uncertainty_B = U2
    
    return(NAPS_sample)

#=============================================================================#
#--------------------------| Tensorflow for LR |------------------------------#
#=============================================================================#
 
# num_classes = 10 # 0 to 9 digits
# # num_features = 784 # 28*28
# learning_rate = 0.01
# training_steps = 1000
# batch_size = 256
# display_step = 50

# W = tf.Variable(tf.ones([num_features, num_classes]), name="weight")
# b = tf.Variable(tf.zeros([num_classes]), name="bias")

# def logistic_regression(x):
#     # Apply softmax to normalize the logits to a probability distribution.
#     return tf.nn.softmax(tf.matmul(x, W) + b)

# def cross_entropy(y_pred, y_true):
#     # Encode label to a one hot vector.
#     y_true = tf.one_hot(y_true, depth=num_classes)
#     # Clip prediction values to avoid log(0) error.
#     y_pred = tf.clip_by_value(y_pred, 1e-9, 1.)
#     # Compute cross-entropy.
#     return tf.reduce_mean(-tf.reduce_sum(y_true * tf.math.log(y_pred)))

# def accuracy(y_pred, y_true):
#     # Predicted class is the index of the highest score in prediction vector (i.e. argmax).
#     correct_prediction = tf.equal(tf.argmax(y_pred, 1), tf.cast(y_true, tf.int64))
#     return tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# # Stochastic gradient descent optimizer.
# optimizer = tf.optimizers.SGD(learning_rate)

if __name__ == '__main__':
    
    
    #=============================================================================#
    #-------------------------------| INPUTS |------------------------------------#
    #=============================================================================#
    start_time = timeit.default_timer()
    
    
    #=============================================================================#
    #-------------------------| Reading in the data |-----------------------------#
    #=============================================================================#
    
    
    Activities, Sensors = Set_Act_Sens() #creating two dicts for sensor and activity
    
    print('\n#------------- Reading in the Data of Users -------------#\n')
    dataset_uuid = readdata_csv(data_dir) #reading all data and storing in "dataset" a DF
    stop1 = timeit.default_timer()
    
    print('Reading the data took:   ', int(stop1 - start_time))
    
    uuids = list(dataset_uuid.keys())
    
    print('\n#-------------- Combining the Data of Users-------------#\n')
    
    #We concatenate the data of all participants (60) to get one dataset for all      
    dataset_ag = dataset_uuid[uuids[0]] #"dataset_ag" is the aggregation of all user's data
    for i in range(1,len(uuids)):
        dataset_ag = pd.concat([dataset_ag, dataset_uuid[uuids[i]]], axis=0, ignore_index=True)
    
    dataset_ag.iloc[:,feature_range] = preprocessing.scale(dataset_ag.iloc[:,feature_range])
    stop2 = timeit.default_timer()
    
    print('Combining the data took:   ', int(stop2 - stop1))
    
    #=============================================================================#
    #-----------------------------| DST Setups |----------------------------------#
    #=============================================================================#
    
    #We create feature sets, a sample mass function (initialized to 0) and response
    #permutations 1 and 2 in which corresponding elements are exclusive and exhaustive
    
    feature_sets = feature_set(sensors_to_fuse, feature_sets_st, Sensors, feature_sets_count)
    mass_template = BPA_builder(FOD)
    Response_Perm_1, Response_Perm_2 = pair_resp(mass_template)
    
    num_p = len(FOD)
    num_fs = len(feature_sets)
    num_rp = len(Response_Perm_1)
    num_folds = 5
    
    smt = SMOTE()
    
    #find the train_dataset
    #at personal level:
    print('\n#-------------- Obtaining Training Dataset -------------#\n')
    
    train_dataset, test_dataset = train_test_spl(0,num_folds,cvdir,dataset_uuid)
    
    stop3 = timeit.default_timer()
    print('Obtaining the training dataset took:   ', int(stop3 - stop2))
    
    print('Training dataset has  {}  samples'.format(len(train_dataset)))
    
    
    #=============================================================================#
    #------------------| Creating and Training all the models |-------------------#
    #=============================================================================#
    
    #------------------------ Parallelization goes here  -------------------------#
    
    #impute = True
    #NAPS_Model = []
    #with Pool(processes= num_prc) as pool:
    #    pool.map(f, [1,2,3])
    #
    #if __name__ == '__main__':
    #    # start 4 worker processes
    #    with Pool(processes=4) as pool:
    #
    #        # print "[0, 1, 4,..., 81]"
    #        print(pool.map(f, range(10)))
    #    NAPS_Model += pool.map(NAPS_Models_Trainer, (num_rp, fs_range, train_dataset\
    #                                                  , feature_sets, Response_Perm_1\
    #                                                  , Response_Perm_2, impute))
    
    #NAPS_Models = NAPS_Models_Trainer(num_rp, fs_range, train_dataset, feature_sets\
    #                                  , Response_Perm_1, Response_Perm_2, impute = True)
    
    print('\n#-------------- Creating and Training Models -------------#\n')
    
    NAPS_models = []
    print('\nLooping over Response Permutations \n ')
    
    for i in range(num_rp): #i runs over response permutations
        
        start_rp = timer()
        
        print('\nResponse Permutation {}/{}'.format(i+1,num_rp))
        print('\n\tLooping over feature sets')
        NAPS_models.append([])
        
        progress = ProgressBar(num_fs, fmt = ProgressBar.FULL)
        
        for j in range(num_fs): #j runs over feature sets
            NAPS_models[i].append([])
            
            NAPS_models[i][j] = \
                train_NAPS_Models(train_dataset, feature_sets, j, Response_Perm_1, \
                          Response_Perm_2, i, bagging_R, num_bags, impute = True)
            #find X and y
    #         X_train, y_train, y1, y2 = Xy(train_dataset, feature_sets, j, \
    #                         Response_Perm_1, Response_Perm_2, i, impute = True)
    # #        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
            
    #         X_train_tmp, y_train_tmp = smt.fit_resample(X_train, y_train)
            
    #         X_train = pd.DataFrame(X_train_tmp, columns = X_train.columns)
    #         y_train = pd.DataFrame(y_train_tmp, columns = ['Merged_label'])
            
            
    #         if j == 0:
    #             U2 = Uncertainty_Bias([y1, y2])
    
    #         #create a model and train it
    #         NAPS_models[i][j] = DS_Model(Response_Perm_1[i], Response_Perm_2[i], \
    #                    X_train, y_train, j)
    #         NAPS_models[i][j].Bags_Trainer(X_train, y_train, bagging_R, num_bags)
    #         NAPS_models[i][j].Uncertainty_B = U2
            
            progress.current += 1
            progress()
        
        progress.done()
        
        stop_rp = timer()
        
        print("\n It took : ", stop_rp - start_rp)
    
    stop4 = timeit.default_timer()
    print('Training all models took:   ', int(stop4 - stop3))
        
    #=============================================================================#
    #------------------| Model Selection and Testing Models |---------------------#
    #=============================================================================#
    
    print('\n#-------------- Testing the Models -------------#\n')
    test_dataset[FOD] = test_dataset[FOD].fillna(0)
    test_dataset = test_dataset[np.sum(test_dataset[FOD], axis=1) != 0]
    
    y_test_ag = np.zeros([len(test_dataset),len(FOD)])
    y_pred_ag = np.zeros([len(test_dataset),len(FOD)])
    
    
    for t in range(len(test_dataset)):
    # for t in range(50):
        print(t,'/',len(test_dataset))
        test_sample = test_dataset.iloc[t,:]
        test_sample = test_sample.to_frame().transpose()
        y_test_ag[t,:] = np.floor(test_sample[FOD].fillna(0).values)
        
        assert np.sum(y_test_ag == 1)
        
        Uncertainty_Mat = np.ones([num_rp, num_fs])
        
        for i in range(num_rp):
            for j in range(num_fs):
                X_test, y_test, y1, y2 = Xy(test_sample, feature_sets, j, \
                                        Response_Perm_1, Response_Perm_2, i, impute = True)
                if len(X_test) != 0 or len(y_test) != 0:
                    Uncertainty_Mat[i][j] = (NAPS_models[i][j].Uncertainty_B +\
                                    NAPS_models[i][j].Uncertainty_Context(X_test))/2
                NAPS_models[i][j].Mass_Function_Setter(Uncertainty_Mat[i][j], X_test)
                
        #=========\ Model Selection/==========#
        
        Selected_Models_idx = Model_Selector(Uncertainty_Mat, models_per_rp, num_rp, 1)
        y_pred_ag[t,:] = Fuse_and_Predict(Selected_Models_idx, NAPS_models, FOD, num_p, \
                  num_rp, models_per_rp)
    
    stop5 = timeit.default_timer()
    print('Testing took:   ', int(stop5 - stop4))
    
    
    
    #conf_mat = confusion_matrix(y_test_ag, y_pred_ag)
    accuracy = accuracy_score(y_test_ag, y_pred_ag)
    #balanced_accuracy = balanced_accuracy_score(y_test_ag, y_pred_ag)
    f1 = f1_score(y_test_ag, y_pred_ag, average='weighted')
    
    
    
    #!!!!!!!!! Model Selection based on the uncertainty should be fixed !!!!!!!!
    


#------------- Reading in the Data of Users -------------#

User 1/60  -> Shape of the data     (2287, 278)
User 2/60  -> Shape of the data     (6813, 278)
User 3/60  -> Shape of the data     (3960, 278)
User 4/60  -> Shape of the data     (3108, 278)
User 5/60  -> Shape of the data     (7521, 278)
User 6/60  -> Shape of the data     (2685, 278)
User 7/60  -> Shape of the data     (8845, 278)
User 8/60  -> Shape of the data     (6218, 278)
User 9/60  -> Shape of the data     (6549, 278)
User 10/60  -> Shape of the data     (7375, 278)
User 11/60  -> Shape of the data     (4771, 278)
User 12/60  -> Shape of the data     (4927, 278)
User 13/60  -> Shape of the data     (8516, 278)
User 14/60  -> Shape of the data     (6172, 278)
User 15/60  -> Shape of the data     (5203, 278)
User 16/60  -> Shape of the data     (7649, 278)
User 17/60  -> Shape of the data     (6691, 278)
User 18/60  -> Shape of the data     (3250, 278)
User 19/60  -> Shape of the data     (4979, 278)
User 20/60  -> Sh

KeyboardInterrupt: 

In [2]:
import time as tm
import multiprocessing as mp
def f(x):
    return x**2-1

f(5)

24

In [ ]:
if __name__ == '__main__':
    start = tm.time()
    with mp.Pool(processes=1) as pool:
        print(pool.map(f, range(10)))

    end = tm.time()
    print(f'processing time is {end-start}')